In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

def factorial_anova(measurements, factors, levels, replicates):
    
    # Create a dictionary to store the data for each factor level combination
    data_dict = {}
    
    # Loop over each factor and its levels to create the data for the ANOVA
    for factor, levels_list in zip(factors, levels):
        for level in levels_list:
            
            # Create a list of measurements for the current factor level combination
            data = []
            for i in range(replicates):
                data.extend([measurements[j] for j in range(len(measurements)) if (j % (len(levels_list)*replicates))//replicates == levels_list.index(level)])
            
            # Store the data in the data dictionary
            data_dict[factor + str(level)] = data
    print(data_dict)
    # Create a Pandas dataframe from the data dictionary
    df = pd.DataFrame.from_dict(data_dict)
    
    # Use the ols function from statsmodels to fit an ANOVA model to the data
    formula = ' + '.join([factor for factor in df.columns])
    model = ols('value ~ ' + formula, data=df).fit()
    anova_table = sm.stats.anova_lm(model, typ=2)
    
    # Extract the sum of squares, mean squares, and F-measure for each factor and factor iteration
    ss_dict = {}
    ms_dict = {}
    f_dict = {}
    for factor in factors:
        ss_dict[factor] = anova_table['sum_sq'][factor]
        ms_dict[factor] = anova_table['mean_sq'][factor]
        f_dict[factor] = anova_table['F'][factor]
        
        for level in levels[factors.index(factor)]:
            ss_dict[factor + str(level)] = anova_table['sum_sq'][factor + str(level)]
            ms_dict[factor + str(level)] = anova_table['mean_sq'][factor + str(level)]
            f_dict[factor + str(level)] = anova_table['F'][factor + str(level)]
    
    # Return the ANOVA metrics as a dictionary
    return {'sum_of_squares': ss_dict, 'mean_squares': ms_dict, 'F_measure': f_dict, 'df': anova_table['df'], 'SS_error': anova_table['sum_sq']['Residual'], 'MS_error': anova_table['mean_sq']['Residual'], 'F_error': anova_table['F']['Residual'], 'p_value': anova_table['PR(>F)']}


In [2]:
measurements = [2, 3, 5, 6, 8, 9, 11, 12]
factors = ['factor1', 'factor2']
levels = [[1, 2], [1, 2]]
replicates = 2

anova_results = factorial_anova(measurements, factors, levels, replicates)
print(anova_results)

{'factor11': [2, 3, 8, 9, 2, 3, 8, 9], 'factor12': [5, 6, 11, 12, 5, 6, 11, 12], 'factor21': [2, 3, 8, 9, 2, 3, 8, 9], 'factor22': [5, 6, 11, 12, 5, 6, 11, 12]}


PatsyError: Error evaluating factor: NameError: name 'value' is not defined
    value ~ factor11 + factor12 + factor21 + factor22
    ^^^^^